In [1]:
# set the matplotlib backend so figures can be saved in the background
import matplotlib
matplotlib.use("Agg")

In [2]:
!pip install tf-nightly-gpu

     |████████████████████████████████| 394.2MB 41kB/s 
     |████████████████████████████████| 14.5MB 237kB/s 
     |████████████████████████████████| 471kB 52.9MB/s 
     |████████████████████████████████| 1.3MB 56.2MB/s 
     |████████████████████████████████| 10.6MB 49.1MB/s 
ERROR: tensorflow 2.3.0 has requirement numpy<1.19.0,>=1.16.0, but you'll have numpy 1.19.2 which is incompatible.
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
  Found existing installation: numpy 1.18.5
    Uninstalling numpy-1.18.5:
      Successfully uninstalled numpy-1.18.5
  Found existing installation: protobuf 3.12.4
    Uninstalling protobuf-3.12.4:
      Successfully uninstalled protobuf-3.12.4


In [3]:
# import the necessary packages
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.applications import MobileNetV3Large
from tensorflow.keras.applications import MobileNetV3Small
from tensorflow.keras.applications import ResNet50

from tensorflow.keras.layers import GlobalAveragePooling2D, Conv2D
from tensorflow.keras.layers import Dropout, Flatten, Dense, Input
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import SGD, RMSprop, Adagrad, Nadam, Adam
from sklearn.metrics import classification_report
from imutils import paths
import matplotlib.pyplot as plt
import numpy as np
import os
import tensorflow as tf

In [4]:
from google.colab import drive
drive.mount('/content/drive')
#drive.mount("/content/drive", force_remount=True)

Mounted at /content/drive


In [5]:
# directory creation
if not os.path.exists("drive/My Drive/colab/output"):
    os.makedirs("drive/My Drive/colab/output")

if not os.path.exists("drive/My Drive/colab/model"):
    os.makedirs("drive/My Drive/colab/model")

In [69]:
BASE_PATH = "drive/My Drive/colab/bed_dataset"
WARMUP_PLOT_PATH = os.path.sep.join(["drive/My Drive/colab/output", "warmup.png"])
UNFROZEN_PLOT_PATH = os.path.sep.join(["drive/My Drive/colab/model", "unfrozen.png"])

# VALUE INIT
IMG_SIZE = 224
IMG_SHAPE = (IMG_SIZE, IMG_SIZE, 3)
base_learning_rate = 0.001
BATCH_SIZE = 32
initial_epochs = 5
fine_tune_epochs = 10
#total_epochs = initial_epochs - fine_tune_epochs
total_epochs = fine_tune_epochs

# initialize the list of class label names
CLASSES = ["clean", "unclean"]

In [70]:
def plot_training(history, N, plotPath):
    # construct a plot that plots and saves the training history
    plt.style.use("ggplot")
    plt.figure()
    plt.plot(np.arange(0, N), history.history["loss"], label="train_loss")
    plt.plot(np.arange(0, N), history.history["val_loss"], label="val_loss")
    #plt.plot(np.arange(0, N), history.history["accuracy"], label="train_acc")
    #plt.plot(np.arange(0, N), history.history["val_accuracy"], label="val_acc")
    plt.title("Training Loss and Accuracy")
    plt.xlabel("Epoch #")
    plt.ylabel("Loss/Accuracy")
    plt.legend(loc="lower left")
    plt.savefig(plotPath)

In [71]:
# derive the paths to the training, validation, and testing
# directories
trainPath = os.path.sep.join([BASE_PATH, "train"])
valPath = os.path.sep.join([BASE_PATH, "val"])
testPath = os.path.sep.join([BASE_PATH, "test"])

# determine the total number of image paths in training, validation, 
# and testing directories
totalTrain = len(list(paths.list_images(trainPath)))
totalVal = len(list(paths.list_images(valPath)))
totalTest = len(list(paths.list_images(testPath)))

In [72]:
# initialize the training data augmentation object
train_datagen = ImageDataGenerator(
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.15,
    zoom_range=0.15,
    horizontal_flip=True,
    fill_mode="nearest")

# initialize the validation/testing data augmentation object (which
# we'll be adding mean subtraction to)
val_datagen = ImageDataGenerator()

# define the ImageNet mean subtraction (in RGB order) and set the
# mean subtraction value for each of the data augmentation
# objects
mean = np.array([123.68, 116.779, 103.939], dtype="float32")
train_datagen.mean = mean
val_datagen.mean = mean

In [73]:
# initialize the training generator
train_batches = train_datagen.flow_from_directory(
    trainPath,
    target_size=(IMG_SIZE, IMG_SIZE),
    color_mode="rgb",
    shuffle=True,
    batch_size=BATCH_SIZE,
    class_mode="categorical") # categorical or binary

# initialize the validation generator
validation_batches = val_datagen.flow_from_directory(
    valPath,
    target_size=(IMG_SIZE, IMG_SIZE),
    color_mode="rgb",
    shuffle=False,
    batch_size=BATCH_SIZE,
    class_mode="categorical")

# initialize the testing generator
test_batches = val_datagen.flow_from_directory(
    testPath,
    target_size=(IMG_SIZE, IMG_SIZE),
    color_mode="rgb",
    shuffle=False,
    batch_size=BATCH_SIZE,
    class_mode="categorical")

Found 213 images belonging to 2 classes.
Found 40 images belonging to 2 classes.
Found 40 images belonging to 2 classes.


In [74]:
# Create the base model from the pre-trained model 
#base_model = MobileNetV2(input_tensor=Input(shape=IMG_SHAPE), weights="imagenet", include_top=False, layers=tf.keras.layers) # added layers=tf.keras.layers for batch normalization problem: still not working
#base_model = ResNet50(input_tensor=Input(shape=IMG_SHAPE), weights="imagenet", include_top=False, layers=tf.keras.layers) # added layers=tf.keras.layers for batch normalization problem: still not working

#base_model = MobileNetV3Large(input_tensor=Input(shape=IMG_SHAPE), weights="imagenet", include_top=False, layers=tf.keras.layers, alpha=1.0, minimalistic=True, backend=tf.keras.backend, models=tf.keras.models, utils=tf.keras.utils)
base_model = MobileNetV3Large(input_tensor=Input(shape=IMG_SHAPE), weights="imagenet", include_top=False, alpha=1.0, minimalistic=True)
# add a global spatial average pooling layer
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Flatten(name="flatten")(x)
# let's add a fully-connected layer
#x = Dense(512, activation="relu")(x) # VGG16
x = Dense(512, activation="relu")(x) # mobilenetv2
x = Dropout(0.8)(x)
# and a logistic layer -- let's say we have config2.CLASSES classes
predictions = Dense(len(CLASSES), activation="softmax")(x)


# this is the model we will train
model = Model(inputs=base_model.input, outputs=predictions)

In [75]:
# first: train only the top layers (which were randomly initialized)
# i.e. freeze all convolutional InceptionV3 layers
#for layer in base_model.layers:
#    layer.trainable = False
    #if isinstance(layer, tf.keras.layers.BatchNormalization):
    #if isinstance(layer, tf.python.keras.layers.normalization_v2.BatchNormalization):
        #layer._per_input_updates = {}

# loop over the layers in the model and show which ones are trainable
# or not

base_model.trainable = True

# Fine-tune from this layer onwards
fine_tune_at = 50

# for layer in base_model.layers[:fine_tune_at]:
#     if type(layer) != type(base_model.layers[3]):
#         layer.trainable = False

for layer in base_model.layers[:fine_tune_at]:
    layer.trainable = False

for layer in base_model.layers:
    print("{}: {}".format(layer, layer.trainable))

print('This is the number of trainable weights '
      'after freezing the conv base:', len(model.trainable_weights))

<tensorflow.python.keras.engine.input_layer.InputLayer object at 0x7fb0a3195860>: False
<tensorflow.python.keras.layers.preprocessing.image_preprocessing.Rescaling object at 0x7fb0a3195b00>: False
<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7fb09f5a9a90>: False
<tensorflow.python.keras.layers.normalization_v2.BatchNormalization object at 0x7fb0d6716a20>: False
<tensorflow.python.keras.layers.advanced_activations.ReLU object at 0x7fb0a3bf7c88>: False
<tensorflow.python.keras.layers.convolutional.DepthwiseConv2D object at 0x7fb0a0216b38>: False
<tensorflow.python.keras.layers.normalization_v2.BatchNormalization object at 0x7fb0a00a31d0>: False
<tensorflow.python.keras.layers.advanced_activations.ReLU object at 0x7fb0a00a3438>: False
<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7fb0a311ff98>: False
<tensorflow.python.keras.layers.normalization_v2.BatchNormalization object at 0x7fb0a3d9b978>: False
<tensorflow.python.keras.layers.merge.Add object at

In [76]:
print("[INFO] compiling model...")
#opt = SGD(lr=base_learning_rate, momentum=0.9, decay=base_learning_rate/initial_epochs)
opt = Adam(lr=base_learning_rate, decay=base_learning_rate/initial_epochs)
#opt = RMSprop(lr=base_learning_rate, decay=base_learning_rate/initial_epochs)

model.compile(loss="binary_crossentropy", optimizer=opt, metrics=["accuracy"]) #binary_crossentropy  categorical_crossentropy

# train the head of the network for a few epochs (all other layers
# are frozen) -- this will allow the new FC layers to start to become
# initialized with actual "learned" values versus pure random
print("[INFO] training head...")
history = model.fit(train_batches,
    epochs=initial_epochs,
    validation_data=validation_batches,
    steps_per_epoch=totalTrain // BATCH_SIZE,
    validation_steps=totalVal // BATCH_SIZE)

# reset the testing generator and evaluate the network after
# fine-tuning just the network head
print("[INFO] evaluating after fine-tuning network head...")
test_batches.reset()
predIdxs = model.predict(x=test_batches, steps=(totalTest // BATCH_SIZE) + 1)
predIdxs = np.argmax(predIdxs, axis=1)
print(classification_report(test_batches.classes, predIdxs, target_names=test_batches.class_indices.keys()))
#plot_training(history, initial_epochs, WARMUP_PLOT_PATH)
model.save("model.h5", save_format="h5")

[INFO] compiling model...
[INFO] training head...
Epoch 1/5
6/6 [==============================] - 22s 3s/step - loss: 1.5282 - accuracy: 0.4605 - val_loss: 0.4404 - val_accuracy: 0.9062
Epoch 2/5
6/6 [==============================] - 18s 3s/step - loss: 0.6050 - accuracy: 0.8095 - val_loss: 0.2044 - val_accuracy: 0.9062
Epoch 3/5
6/6 [==============================] - 17s 3s/step - loss: 0.2567 - accuracy: 0.9409 - val_loss: 0.2150 - val_accuracy: 0.8750
Epoch 4/5
6/6 [==============================] - 18s 3s/step - loss: 0.2013 - accuracy: 0.9319 - val_loss: 0.2357 - val_accuracy: 0.9062
Epoch 5/5
6/6 [==============================] - 18s 3s/step - loss: 0.1628 - accuracy: 0.9306 - val_loss: 0.2084 - val_accuracy: 0.9062
[INFO] evaluating after fine-tuning network head...
              precision    recall  f1-score   support

       clean       0.77      1.00      0.87        20
     unclean       1.00      0.70      0.82        20

    accuracy                           0.85      

In [30]:
history_dict = history.history
print(history_dict.keys())

dict_keys(['loss', 'accuracy'])


In [35]:
type(base_model.layers[3]) == type(base_model.layers[6])

True

In [77]:
model.save("model.h5", save_format="h5")

In [78]:
!cp model.h5 "drive/My Drive/colab/model/model.h5"